## PROJECT-3. Решение комплексной бизнес-задачи

### Цель:
-  создать рекомендательную систему по курсам, чтобы организовать их допродажу для увеличения среднего чека. 

### Задачи:
- Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы. 
- Подготовить и проанализировать имеющиеся данные.
- Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить  продакт-менеджеру.

### План:
- Знакомство с датасетом.
- Подготовка данных (при помощи SQL).
- Обработка данных (при помощи Python).
- Составление отчета (таблицу с рекомендациями)

--С помощью SQL-запроса выведем данные с id-пользователей и id, приобретенных ими курсов. Возьмем данные только по тем пользователям, которые приобрели два и более курса.--

with user_resource as (
  
  select 
    
    distinct c.user_id, 
    
    ci.resource_id 
  
  from 
    
    final.cart_items ci 
    
    join final.carts c on ci.cart_id = c.id 
  
  where 
    
    resource_type = 'Course' 
    
    and c.state = 'successful'

), 

user_count_resource as (
  
  select 
    
    ur.user_id, 
    
    count(ur.resource_id) count_resource 
  
  from 
    
    user_resource ur 
  
  group by 
    
    1 
  
  having 
    
    count(ur.resource_id)> 1

) 

select 
  
  ucr.user_id, 
  
  ur.resource_id 

from 
  
  user_count_resource ucr 
  
  join user_resource ur on ucr.user_id = ur.user_id 

order by 
  
  1, 
  
  2


In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import itertools
import collections

In [2]:
# Загрузим данные, полученные с помощью SQL-запроса
courses=pd.read_csv('courses.csv')
courses.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [3]:
# С помощью метода info посмотрим на наши данные (кол-во строк, столбцов, типы данных, наличие пропусков)
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [4]:
# Переименуем столбцы
courses=courses.rename(columns={'user_id':'client_id', 'resource_id':'courses_id'})
courses.head()

,client_id,courses_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [5]:
# Определим количество уникальных пользователей
len(courses['client_id'].unique())

12656

In [6]:
# Определим количество уникальных курсов
len(courses['courses_id'].unique())

126

In [7]:
# Объединим, купленные каждым пользователем курсы, в список
clients_courses_list=courses.groupby(['client_id'])['courses_id'].apply(lambda x: np.unique(x)).reset_index()
clients_courses_list.head()

,client_id,courses_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [8]:
# С помощью метода info посмотрим на наши данные (кол-во строк, столбцов, типы данных, наличие пропусков)
clients_courses_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12656 entries, 0 to 12655
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   client_id   12656 non-null  int64 
 1   courses_id  12656 non-null  object
dtypes: int64(1), object(1)
memory usage: 197.9+ KB


In [9]:
# Создадим один общий список всех возможных комбинаций пар курсов с помощью метода itertools.combinations
courses_pairs=[]
for course in clients_courses_list['courses_id']:
    for pair in itertools.combinations(sorted(course), 2):
        courses_pairs.append(pair)

In [10]:
# Посмотрим  сколько уникалных пар курсов
len(set(courses_pairs))

3989

In [11]:
# С помощью функции Counter посчитаем частоту встречаемости пар курсов
count_pairs=collections.Counter()
for i in courses_pairs:
    count_pairs[i]+=1

In [12]:
#Создадим датафрейм с парами курсов
df_courses=pd.DataFrame(courses_pairs)

#Переименуем столбцы
df_courses=df_courses.rename(columns={0:'course_1', 1:'course_2'})

#Добавим столбец с парой курсов
df_courses['pair']=df_courses['course_1'].astype(str)+","+df_courses['course_2'].astype(str)
df_courses.head()

,course_1,course_2,pair
0,516,1099,"516,1099"
1,356,357,"356,357"
2,356,1125,"356,1125"
3,357,1125,"357,1125"
4,553,1147,"553,1147"


In [13]:
# Посчитаем количество встречаемости каждой пары курсов, сгруппировав данные по парам курсов
group_df_courses=df_courses.groupby('pair').count().reset_index()

# Оставим два необходимых столбца и переименуем столбец с данными о количестве пар.
group_df_courses=group_df_courses[['pair', 'course_1']].rename(columns={'course_1':'count'})
group_df_courses.head()

,pair,count
0,"1099,1100",5
1,"1099,1101",2
2,"1099,1102",2
3,"1099,1103",4
4,"1099,1125",4


In [14]:
# Для того, чтобы отсечь из анализа пары курсов, которые встречаются редко, используем 90-й процентиль. 
# Чаще всего используют 99-й, 95-й и 90-й процентиль (статья habr.com). На мой взгляд, необходимо использовать именно 90-й 
# процентиль, т.к. иначе в таблице рекомендаций будет очень много пропусков.

# Если взять 90-й процентиль (останется не пустых значений 55(рекоменд_1) и 44(рекоменд_2) из 126)
# 95-й процентиль (останется не пустых значений 44 и 30 из 126)
# 99-й процентиль (останется не пустых значений 17 и 11 из 126 )

percentile=np.percentile(group_df_courses['count'], 90)
percentile

22.0

In [15]:
# Создадим новый словарь, в котором не будет пар курсов, встречающихся реже 22 раз
# Отсортируем пары по 1 курсу
popular_pairs={k:v for k, v in sorted(count_pairs.items(), key=lambda item: item[0], reverse=False) if v>percentile}

In [16]:
 # Создадим функцию
def recommendation(course): 
    course_list=[]          # Создадим пустой список
    for i in popular_pairs.keys(): # Создадим цикл и пройдем по ключам словаря с популярными парами курсов
        if i[0]==course:  # Если 1-й элемент пары курсов равен id курса, который мы подаем на вход ф-ии,
            course_list.append((i, popular_pairs[i])) # то добавляем в список кортеж из пары курсов и количества
    sorted_course_list=sorted(course_list, key=lambda x: x[1], reverse=True) # Сортируем список в порядке убывания по кол-ву
    return sorted_course_list[:2] # Выводим два наибольших значения, которые потом включим в рекомендации №1 и №2

In [17]:
# Создадим переменную, в которой хранятся данные с уникальными курсами
courses_1=courses['courses_id'].unique()
courses_1

array([ 516, 1099,  356,  357, 1125,  553, 1147,  361, 1138, 1140,  551,
        745,  568,  514,  517,  566,  363,  511,  562,  563,  509, 1144,
        672,  552,  571,  513, 1141,  744,  862,  679,  750,  800,  569,
        840,  765, 1187, 1100, 1103,  502,  564,  865,  764, 1139, 1186,
        366,  367,  519,  809,  515,  912,  489,  523,  864, 1101, 1146,
        776,  671,  753,  829,  490, 1102,  803,  659,  909,  794,  518,
        907,  777,  908,  360,  813,  835,  741,  752,  814, 1115, 1116,
       1161,  863,  743,  504,  572,  810, 1124, 1128,  742, 1104,  503,
        664,  507,  570, 1185, 1198,  365,  359,  791, 1156,  362, 1184,
        911,  358, 1160,  757,  508, 1181,  755, 1145, 1188,  756,  866,
        749,  368,  364,  834, 1152,  670, 1199,  836, 1201, 1129, 1182,
        902,  837, 1200,  833,  830], dtype=int64)

In [18]:
# Создадим таблицу с курсами и рекомендациями
recommendation_df=pd.DataFrame(columns=['recommendation_1','recommendation_2'])
for i in courses_1:
    if len(recommendation(i))==2: # Если функция выдает две рекомендации
            recommendation_df.loc[i]=[recommendation(i)[0][0][1], recommendation(i)[1][0][1]]# Добавляем рекомендации в табл
    elif len(recommendation(i))==1: # Если функция выдает одну рекомендацию
         recommendation_df.loc[i]=[recommendation(i)[0][0][1], np.nan] # Добавляем 1 рекомендацию и NaN вместо 2-й рекоменд.
    else:
        recommendation_df.loc[i]=[np.nan, np.nan] # Добавляем NaN, если нет рекомендаций
recommendation_df=recommendation_df.reset_index().rename(columns={'index':'course_id'})# Сбросим индексы,переименуем столбцы
recommendation_df

,course_id,recommendation_1,recommendation_2
0,516,745,553
1,1099,NaN,NaN
2,356,571,357
3,357,571,1125
4,1125,1186,NaN
...,...,...,...
121,902,NaN,NaN
122,837,NaN,NaN
123,1200,NaN,NaN
124,833,NaN,NaN


In [19]:
# Посмотрим сколько у нас пустых строк
recommendation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   course_id         126 non-null    int64 
 1   recommendation_1  55 non-null     object
 2   recommendation_2  44 non-null     object
dtypes: int64(1), object(2)
memory usage: 3.1+ KB


In [20]:
# Отсортируем датафрейм по столбцу course_id
recommendation_df=recommendation_df.sort_values("course_id").reset_index(drop=True)
recommendation_df.head()

,course_id,recommendation_1,recommendation_2
0,356,571,357
1,357,571,1125
2,358,570,NaN
3,359,NaN,NaN
4,360,NaN,NaN


### Чем заполнить пропуски?
##### В полученной таблице с рекомендациями есть пустые значения, которые необходимо заполнить:
1. Изучим данные продаж курсов в разрезе пользователей, купивших два и более курса
2. Рассмотрим несколько возможных вариантов:
- добавим вместо пропусков наиболее популярные курсы (без учета новые они или давно в продаже);
- добавим вместо пропусков наиболее популярные из новых курсов;
- добавим вместо пропусков один наиболее популярный курс, который давно в продаже и один новый курс.

#### Рассмотрим, какие курсы самые популярные у пользователей, купивших 2 и более курсов 

In [21]:
# Сгруппируем данные по id-курса и посчитаем количество купленных курсов за два года (нам даны данные за 2017-2018гг.)  
courses_popul=courses.groupby('courses_id').count().reset_index()
courses_popul=courses_popul.rename(columns={'client_id':'count_courses'})
courses_popul=courses_popul.sort_values('count_courses', ascending=False).reset_index(drop=True)
courses_popul.head(8)

,courses_id,count_courses
0,551,2935
1,566,2342
2,515,1311
3,489,1125
4,490,1084
5,523,1053
6,514,983
7,794,875


- Если следовать данному подходу, то нужно пропуски в столбце recommendation_1 заменить на курс с id-551, а в столбце recommendation_2 - на курс с id-566 

#### Рассмотрим, какие курсы самые популярные у пользователей, купивших 2 и более курсов с РАЗБИВКОЙ по ГОДАМ продажи

In [22]:
# Загрузим данные
courses_2_2017_2018=pd.read_csv('courses_2_2017_2018.csv')
courses_2_2017_2018

,resource_id,count_purchased,year_purchased
0,551,2801,2017
1,566,1191,2018
2,566,1160,2017
3,515,1034,2017
4,794,888,2018
...,...,...,...
186,911,3,2018
187,833,3,2018
188,1199,2,2018
189,1201,2,2018


In [23]:
courses_2_2017_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   resource_id      191 non-null    int64
 1   count_purchased  191 non-null    int64
 2   year_purchased   191 non-null    int64
dtypes: int64(3)
memory usage: 4.6 KB


In [24]:
# Посмотрим данные по продажам за 2017 год
courses_2_2017=courses_2_2017_2018[courses_2_2017_2018['year_purchased']==2017]
courses_2_2017

,resource_id,count_purchased,year_purchased
0,551,2801,2017
2,566,1160,2017
3,515,1034,2017
5,523,875,2017
6,489,857,2017
...,...,...,...
158,671,25,2017
159,364,24,2017
162,365,19,2017
163,670,19,2017



- В 2017 году самыми популярным курсом был курс с номером id - 551, на втором месте - 566

In [25]:
# Посмотрим какие курсы продавались в 2017 году (отсортируем массив по id курсов)
print(np.array(courses_2_2017['resource_id'].sort_values()))

[356 357 358 359 360 361 362 363 364 365 366 367 368 489 490 502 503 504
 507 508 509 511 513 514 515 516 517 518 519 523 551 552 553 562 563 564
 566 568 569 570 571 572 659 664 670 671 672 679 741 742 743 744 745 749
 750 752 753 755 756 757 764 765 776 777 791]


- В 2017 году продавались курсы с номерами id от 356 до 791 

In [26]:
# Рассмотрим данные по продажам за 2018 год
courses_2_2018=courses_2_2017_2018[courses_2_2017_2018['year_purchased']==2018]
courses_2_2018.head(10)

,resource_id,count_purchased,year_purchased
1,566,1191,2018
4,794,888,2018
8,809,640,2018
9,840,615,2018
14,1103,416,2018
15,514,414,2018
17,490,387,2018
20,1100,362,2018
21,1125,354,2018
22,764,344,2018


- в 2018 году наиболее популярный курс с id - 566, на втором месте - 794

In [27]:
# Посмотрим какие продажи в 2018 году курса с id-551, который был самым популярным в 2017г
courses_2_2018[courses_2_2018['resource_id']==551]

,resource_id,count_purchased,year_purchased
74,551,143,2018


- Самый популярный курс в 2018 году с id-551, в 2018 году оказался на 75 месте

In [28]:
### Посмотрим какие курсы продавались в 2018 году (отсортируем DataFrame по id курсов)
print(np.array(courses_2_2018['resource_id'].sort_values()))

[ 356  357  358  359  360  361  362  363  364  365  366  367  368  489
  490  502  503  504  507  508  509  511  513  514  515  516  517  518
  519  523  551  552  553  562  563  564  566  568  569  570  571  572
  659  664  670  671  672  679  741  742  743  744  745  749  750  752
  753  755  756  757  764  765  776  777  791  794  800  803  809  810
  813  814  829  830  833  834  835  836  837  840  862  863  864  865
  866  902  907  908  909  911  912 1099 1100 1101 1102 1103 1104 1115
 1116 1124 1125 1128 1129 1138 1139 1140 1141 1144 1145 1146 1147 1152
 1156 1160 1161 1181 1182 1184 1185 1186 1187 1188 1198 1199 1200 1201]


- В 2018 году продавались курсы с номерами id от 356 до 1201
- В 2018 году появились новые курсы с id от 794 до 1201
- Самый продаваемый курс из новых курсов в 2018 году - 794
- Курс 566 удерживает лидирующие позиции (в 2017 году-2 место, в 2018 - 1 место), а курс 551, который лидирует по общим продажам, в 2017 году был на 1 месте, его продажи были почти в 2,5 раза больше, чем у курса 566, в 2018 году практически не продается (143 продажи, 75 место в рейтинге).

Если анализировать данные по продажам отдельно за 2017 и 2018 годы, и руководствоваться просьбой продакт-менеджера, увеличить продажи новых курсов, то целесообразно, на мой взгляд:
- заменить пропуски в столбце recommendation_1 на курс с id-566 (он второй по популярности за весь период продаж, но зато он стабильно продавался и в 2017, и в 2018г., в отличии от курса с id-551, который является самым популярным, но в 2018г продажи резко упали и в рейтинге он оказался на 75 месте), 
- а в столбце recommendation_2 - на курс с id-794 (в общем объеме продаж за два года он 8-й по популярности, а в 2018г - 2-й по популярности, это объясняется тем, что это новый курс, который появился только в 2018 году и из новых курсов он самый популярный).

In [29]:
# Перед тем, как заполнить пропуски, проверим, есть ли пропуски у курса с id-794
recommendation_df[recommendation_df['course_id']==794]

,course_id,recommendation_1,recommendation_2
65,794,809,1103


In [30]:
# Проверим, есть ли пропуски у курса с id-566
recommendation_df[recommendation_df['course_id']==566]

,course_id,recommendation_1,recommendation_2
36,566,794,570


- Т.к. у курсов 566 и 794 все рекомендации заполнены, мы можем не переживать, что при заполнении пропусков курсы задублируются.

In [31]:
# Заменим пропуски в столбце recommendation_1 на курс с id 566,
# в столбце recommendation_2 - на курс с id 794
recommendation_df['recommendation_1']=recommendation_df['recommendation_1'].fillna(566).astype(int)
recommendation_df['recommendation_2']=recommendation_df['recommendation_2'].fillna(794).astype(int)
recommendation_df

,course_id,recommendation_1,recommendation_2
0,356,571,357
1,357,571,1125
2,358,570,794
3,359,566,794
4,360,566,794
...,...,...,...
121,1188,566,794
122,1198,566,794
123,1199,566,794
124,1200,566,794


In [32]:
# Посмотрим, есть ли курсы, у которых две одинаковые рекомендации
recommendation_df[(recommendation_df['recommendation_1'])==(recommendation_df['recommendation_2'])]

,course_id,recommendation_1,recommendation_2
53,749,794,794
63,777,794,794


In [33]:
# Создадим функцию, которая при дублировании рекомендаций, заменит курс 794 в столбце recommendation_2 на курс с id-366
def fillna(row):
    if row['recommendation_2']==row['recommendation_1']:
        return 366
    else:
        return row['recommendation_2']
recommendation_df['recommendation_2'] = recommendation_df.apply(lambda row: fillna(row), axis=1)  
recommendation_df

,course_id,recommendation_1,recommendation_2
0,356,571,357
1,357,571,1125
2,358,570,794
3,359,566,794
4,360,566,794
...,...,...,...
121,1188,566,794
122,1198,566,794
123,1199,566,794
124,1200,566,794


In [34]:
# Проверим работу функции fillna
recommendation_df[recommendation_df['course_id']==749]

,course_id,recommendation_1,recommendation_2
53,749,794,366


In [35]:
# Посмотрим, после применения ф-ии, есть ли курсы, у которых две одинаковые рекомендации
recommendation_df[(recommendation_df['recommendation_1'])==(recommendation_df['recommendation_2'])]

,course_id,recommendation_1,recommendation_2


In [36]:
#Сохраним файл в формате xls
recommendation_df.to_excel("Recommendation_table.xls", index=False)

### Выводы:
1. Курсы с номерами id-551 и 566 являются наиболее популярными у пользователей, купивших два и более курсов, за весь предоставленный период продаж (2017-2018гг):
- id-551 - 2935 продаж
- id-566 - 2341 продажа
2. В 2017 году самым популярным курсом был курс с номером id-551, на втором месте курс с id-566:
- id-551 - 2801 продажа
- id-566 - 1160 продажа
3. В 2018 году самым популярным курсом был курс с номером id-566, на втором месте-id-794, а продажи курса с id-551 (самый популярный в 2017) упали приблизительно в 20 раз и в рейтинге он оказался на 75 месте.
- id-566 - 1191 продажа
- id-794 - 888 продаж
- id-551 - 143 продажи
4. Курсы с id от 794 до 1201 появились в продаже только в 2018 году

#### Критерии, по которым были включены курсы в рекомендательную таблицу:
1. в рекомендацию-1 включен наиболее популярный курс, который покупают с данным курсом;
2. в рекомендацию-2 включен второй по популярности курс, который приобретают с данным курсом;
3. для курсов, к которым не удалось найти рекомендацию, предложены следующие курсы:
- рекомендация-1 - курс с id-566 (он второй по популярности за весь период продаж, но зато он стабильно продавался и в 2017, и в 2018г., в отличии от курса с id-551, который является самым популярным, но в 2018г продажи резко упали и в рейтинге он оказался на 75 месте)
- рекомендация-2 - курс с id-794 (в общем объеме продаж за два года он 8-й по популярности, а в 2018г - 2-й по популярности, это объясняется тем, что это новый курс, который появился только в 2018 году и из новых курсов он самый популярный).

### Примечание:

Заменяя пропуски в таблице, я руководствовалась просьбой продакт-менеджера увеличить продажи новых курсов, а так же анализировала продажи отдельно за 2017 и 2018 годы. Поэтому в рекомендацию-1 был включен курс с id-566 (второй в рейтинге продаж, но со стабильными продажами в 2017 и 2018), а в рекомендацию-2 - новый курс с id-794 (самый популярный из новых, и второй по популярности в 2018г).

Так же хочется отметить, что по неизвестным мне причинам, самый популярный курс с id-551,  продажи  которого в 2017 году примерно в 2.5 раза превышали второй по популярности курс, в 2018 году почти не пользовался популярностью, его продажи упали приблизительно в 20 раз. 

При необходимости, по решению продакт-менеджера, в рекомендательной таблице могу заменить курс 566 или 794 на курс с id-551.

#### ЗАПУСКАЕМ А/Б-ТЕСТ

Cпустя месяц на сайте реализована новая функциональность с предложением добавить в корзину второй подходящий курс. 

Чтобы оценить эффективность этой функции и качество подбора рекомендаций запускается сплит-тест, где все клиенты случайным образом делятся на контрольную и тестовую группы. Тестовой группе показываются рекомендации, а контрольной — нет.

- До реализации рекомендаций средняя конверсия в покупку второго курса была 3,2%. Мы ожидаем, что ввод рекомендаций сможет поднять её до 4%.
- Определяем минимальный размер выборки для проведения теста при уровне достоверности 95% и статистической мощности 80%.
(Для расчёта используем онлайн-калькулятором)

- Результат: 7900 - минимальный размер выборки для проведения A/Б теста.

#### Оцениваем результаты А/Б-тестирования
Прошло три недели. Каждый из вариантов сплит-теста достиг необходимого размера выборки, необходимо принять решение, был ли ввод рекомендаций успешен.

РЕЗУЛЬТАТЫ:

1. В контрольной группе оказалось 8732 клиента, оформивших заказ, из них 293 купили больше одного курса.

2. В тестовой — 8847 клиентов, из них 347 купили больше одного курса.

#### Чему равно p-значение?

Для расчёта использован онлайн-калькулятор.

- Результат: P-value=0.022

#### Вывод:
Вариант с рекомендациями показал статистически значимо лучший результат, нужно реализовывать его для всех клиентов.